# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [9]:
init_aviat_df = pd.read_csv('AviationData.csv', encoding = 'latin-1', low_memory=False)

In [10]:
init_aviat_df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [11]:
init_aviat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

In [12]:
init_aviat_df.describe()

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,82805.000000,77488.000000,76379.000000,76956.000000,82977.000000
mean,1.146585,0.647855,0.279881,0.357061,5.325440
std,0.446510,5.485960,1.544084,2.235625,27.913634
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000
max,8.000000,349.000000,161.000000,380.000000,699.000000


## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [13]:
print(init_aviat_df['Aircraft.Category'].isna().sum())
print(init_aviat_df['Aircraft.Category'].value_counts())

56602
Aircraft.Category
Airplane             27617
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
Rocket                   1
ULTR                     1
Name: count, dtype: int64


In [14]:
init_aviat_df.head()[['Make', 'Aircraft.Category']]

,Make,Aircraft.Category
0,Stinson,NaN
1,Piper,NaN
2,Cessna,NaN
3,Rockwell,NaN
4,Cessna,NaN


In [15]:
init_aviat_df['Aircraft.Category'].fillna('Airplalne', inplace=True)
init_aviat_df['Aircraft.Category'].value_counts()

C:\Users\bacat\AppData\Local\Temp\ipykernel_27432\1809401179.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  init_aviat_df['Aircraft.Category'].fillna('Airplalne', inplace=True)


Aircraft.Category
Airplalne            56602
Airplane             27617
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
Rocket                   1
ULTR                     1
Name: count, dtype: int64

In [16]:
air_df = init_aviat_df[init_aviat_df['Aircraft.Category'] == 'Airplane']
air_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27617 entries, 5 to 88886
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                27617 non-null  object 
 1   Investigation.Type      27617 non-null  object 
 2   Accident.Number         27617 non-null  object 
 3   Event.Date              27617 non-null  object 
 4   Location                27610 non-null  object 
 5   Country                 27610 non-null  object 
 6   Latitude                22092 non-null  object 
 7   Longitude               22083 non-null  object 
 8   Airport.Code            17676 non-null  object 
 9   Airport.Name            18222 non-null  object 
 10  Injury.Severity         26803 non-null  object 
 11  Aircraft.damage         26335 non-null  object 
 12  Aircraft.Category       27617 non-null  object 
 13  Registration.Number     27382 non-null  object 
 14  Make                    27608 non-null  obj

In [17]:
air_df['Amateur.Built'].value_counts()

Amateur.Built
No     24417
Yes     3183
Name: count, dtype: int64

In [18]:
#filter amateur builds
air_df = air_df[air_df['Amateur.Built'] == 'No']

In [19]:
#removing all events older than 40 yrs ago
air_df = air_df.dropna(subset=['Publication.Date'])
air_df['Report_Date'] = pd.to_datetime(air_df['Publication.Date'])
air_df = air_df.sort_values(by = ['Report_Date'])
air_df = air_df[air_df['Report_Date'] > '1983']

C:\Users\bacat\AppData\Local\Temp\ipykernel_27432\3177713494.py:3: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  air_df['Report_Date'] = pd.to_datetime(air_df['Publication.Date'])


In [20]:
air_df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Report_Date
21,20020917X01776,Accident,CHI82DA020,1982-01-02,"CHARLOTTE, MI",United States,NaN,NaN,49G,TINKERBELL,...,NaN,0.0,0.0,0.0,2.0,VMC,Approach,Probable Cause,02-01-1983,1983-01-02
12,20020917X02148,Accident,FTW82FRJ07,1982-01-02,"HOMER, LA",United States,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,1.0,0.0,IMC,Cruise,Probable Cause,02-01-1983,1983-01-02
13,20020917X02134,Accident,FTW82FRA14,1982-01-02,"HEARNE, TX",United States,NaN,NaN,T72,HEARNE MUNICIPAL,...,NaN,1.0,0.0,0.0,0.0,IMC,Takeoff,Probable Cause,02-01-1983,1983-01-02
14,20020917X02119,Accident,FTW82FPJ10,1982-01-02,"CHICKASHA, OK",United States,NaN,NaN,NaN,NaN,...,NaN,1.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,02-01-1983,1983-01-02
15,20020917X02117,Accident,FTW82FPG08,1982-01-02,"LITTLE ROCK, AR",United States,NaN,NaN,NaN,NaN,...,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,02-01-1983,1983-01-02


In [21]:
air_df['Report_Date'].describe()

count                            23458
mean     2014-07-20 06:00:12.891124480
min                1983-01-02 00:00:00
25%                2019-06-22 18:00:00
50%                2020-09-25 00:00:00
75%                2020-09-25 00:00:00
max                2022-12-30 00:00:00
Name: Report_Date, dtype: object

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [22]:
#explore injury severity in demo then index col list to exclude

injury_col_names = air_df.columns[air_df.columns.str.contains('njur')][1::]
print(injury_col_names)

air_df[injury_col_names].isna().sum()

Index(['Total.Fatal.Injuries', 'Total.Serious.Injuries',
       'Total.Minor.Injuries', 'Total.Uninjured'],
      dtype='object')


Total.Fatal.Injuries      2644
Total.Serious.Injuries    2656
Total.Minor.Injuries      2371
Total.Uninjured            583
dtype: int64

In [23]:
#there are columns with no data in injuries stats and should be removed.

air_df[injury_col_names].isna().all(axis = 1).sum

<bound method Series.sum of 21       False
12       False
13       False
14       False
15       False
         ...  
88886    False
88857    False
88788    False
88708    False
88784    False
Length: 23458, dtype: bool>

In [24]:
air_df = air_df[~air_df[injury_col_names].isna().all(axis = 1)]
air_df[injury_col_names].isna().sum()

Total.Fatal.Injuries      2629
Total.Serious.Injuries    2641
Total.Minor.Injuries      2356
Total.Uninjured            568
dtype: int64

In [25]:
#We will assume for the rest that a blank meant the corresponding data was 0 and was not recorded
#other valid strategies are dropping all NaNs, etc. 

air_df.loc[:,injury_col_names] = air_df[injury_col_names].fillna(0)
air_df[injury_col_names].isna().sum()

Total.Fatal.Injuries      0
Total.Serious.Injuries    0
Total.Minor.Injuries      0
Total.Uninjured           0
dtype: int64

In [26]:
#client is interestd in the likelihood of fatal/serious injuries given an accident.
# this can be represented as a fraction of fatal and serious injuries over the total number of passengers
#(injured + uninjuried)

#calculate the total passenger number
air_df.loc[:,'N_passenger'] = air_df.loc[:,injury_col_names].sum(axis=1)

#remove any rows where via the total num of passengers has been calculated to 0
air_df = air_df[air_df['N_passenger'] > 0]

#calculate fatal and serious injury fraction
air_df['ser_inj_frac'] = (air_df['Total.Fatal.Injuries'] + air_df['Total.Serious.Injuries'])/air_df['N_passenger']

In [27]:
air_df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Report_Date,N_passenger,ser_inj_frac
21,20020917X01776,Accident,CHI82DA020,1982-01-02,"CHARLOTTE, MI",United States,NaN,NaN,49G,TINKERBELL,...,0.0,0.0,2.0,VMC,Approach,Probable Cause,02-01-1983,1983-01-02,2.0,0.0
12,20020917X02148,Accident,FTW82FRJ07,1982-01-02,"HOMER, LA",United States,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,IMC,Cruise,Probable Cause,02-01-1983,1983-01-02,1.0,0.0
13,20020917X02134,Accident,FTW82FRA14,1982-01-02,"HEARNE, TX",United States,NaN,NaN,T72,HEARNE MUNICIPAL,...,0.0,0.0,0.0,IMC,Takeoff,Probable Cause,02-01-1983,1983-01-02,1.0,1.0
14,20020917X02119,Accident,FTW82FPJ10,1982-01-02,"CHICKASHA, OK",United States,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,IMC,Cruise,Probable Cause,02-01-1983,1983-01-02,1.0,1.0
15,20020917X02117,Accident,FTW82FPG08,1982-01-02,"LITTLE ROCK, AR",United States,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,IMC,Cruise,Probable Cause,02-01-1983,1983-01-02,2.0,1.0


**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [28]:
air_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22785 entries, 21 to 88784
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                22785 non-null  object        
 1   Investigation.Type      22785 non-null  object        
 2   Accident.Number         22785 non-null  object        
 3   Event.Date              22785 non-null  object        
 4   Location                22784 non-null  object        
 5   Country                 22778 non-null  object        
 6   Latitude                18532 non-null  object        
 7   Longitude               18526 non-null  object        
 8   Airport.Code            15000 non-null  object        
 9   Airport.Name            15557 non-null  object        
 10  Injury.Severity         22785 non-null  object        
 11  Aircraft.damage         22023 non-null  object        
 12  Aircraft.Category       22785 non-null  object    

In [29]:
print(air_df['Aircraft.damage'].unique())
print(air_df['Aircraft.damage'].value_counts())

['Substantial' 'Destroyed' 'Minor' nan 'Unknown']
Aircraft.damage
Substantial    18415
Destroyed       2917
Minor            650
Unknown           41
Name: count, dtype: int64


In [30]:
air_df['Aircraft.damage'] = air_df['Aircraft.damage'].replace({'Unknown':np.nan})
air_df.dropna(subset=('Aircraft.damage'), inplace=True)
air_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21982 entries, 21 to 88784
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                21982 non-null  object        
 1   Investigation.Type      21982 non-null  object        
 2   Accident.Number         21982 non-null  object        
 3   Event.Date              21982 non-null  object        
 4   Location                21982 non-null  object        
 5   Country                 21975 non-null  object        
 6   Latitude                18060 non-null  object        
 7   Longitude               18053 non-null  object        
 8   Airport.Code            14675 non-null  object        
 9   Airport.Name            15226 non-null  object        
 10  Injury.Severity         21982 non-null  object        
 11  Aircraft.damage         21982 non-null  object        
 12  Aircraft.Category       21982 non-null  object    

In [31]:
#create derived colunm for aircraft destroyed
# (boolean mask / filter and convert to int with 1 = destroyed)

air_df['is_destroyed'] = (air_df['Aircraft.damage'] == 'Destroyed').astype('int')

### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [32]:
air_df.loc[:,'Make'] = air_df['Make'].str.upper()
air_df['Make'].value_counts()

Make
CESSNA                    8022
PIPER                     4517
BEECH                     1606
BOEING                     470
MOONEY                     398
                          ... 
SLIP STREAM                  1
AG-CAT CORPORATION           1
REIMS AVIATION S.A.          1
ARCTIC AIRCRAFT CO INC       1
SCOTT TERRY G                1
Name: count, Length: 1046, dtype: int64

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [33]:
air_df.dropna(subset=['Model'], inplace = True)

In [34]:
# as an example both beech and aero-commander have a model 100
air_df.groupby(['Model', 'Make']).count().loc['100']

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Report_Date,N_passenger,ser_inj_frac,is_destroyed
Make,,,,,,,,,,,,,,,,,,,,,
AERO COMMANDER,15,15,15,15,15,15,11,11,7,11,...,15,15,15,7,13,15,15,15,15,15
BEECH,3,3,3,3,3,3,2,2,1,1,...,3,3,3,1,3,3,3,3,3,3
HAWKER-BEECHCRAFT,1,1,1,1,1,1,1,1,0,0,...,1,1,1,0,1,1,1,1,1,1
OMF,1,1,1,1,1,1,1,1,1,1,...,1,1,0,0,0,1,1,1,1,1


In [35]:
# one possibility is to use multi-indexing another is to merge make-model into a single column
air_df['Make_model'] = air_df['Make'] + '_' + air_df['Model'].str.upper()

In [36]:
air_df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Report_Date,N_passenger,ser_inj_frac,is_destroyed,Make_model
21,20020917X01776,Accident,CHI82DA020,1982-01-02,"CHARLOTTE, MI",United States,NaN,NaN,49G,TINKERBELL,...,2.0,VMC,Approach,Probable Cause,02-01-1983,1983-01-02,2.0,0.0,0,CESSNA_150L
12,20020917X02148,Accident,FTW82FRJ07,1982-01-02,"HOMER, LA",United States,NaN,NaN,NaN,NaN,...,0.0,IMC,Cruise,Probable Cause,02-01-1983,1983-01-02,1.0,0.0,1,BELLANCA_17-30A
13,20020917X02134,Accident,FTW82FRA14,1982-01-02,"HEARNE, TX",United States,NaN,NaN,T72,HEARNE MUNICIPAL,...,0.0,IMC,Takeoff,Probable Cause,02-01-1983,1983-01-02,1.0,1.0,1,CESSNA_R172K
14,20020917X02119,Accident,FTW82FPJ10,1982-01-02,"CHICKASHA, OK",United States,NaN,NaN,NaN,NaN,...,0.0,IMC,Cruise,Probable Cause,02-01-1983,1983-01-02,1.0,1.0,1,NAVION_A
15,20020917X02117,Accident,FTW82FPG08,1982-01-02,"LITTLE ROCK, AR",United States,NaN,NaN,NaN,NaN,...,0.0,IMC,Cruise,Probable Cause,02-01-1983,1983-01-02,2.0,1.0,1,BEECH_19


### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [37]:
print(air_df['Engine.Type'].value_counts())
air_df['Engine.Type'].replace({'Unknown':np.nan, 'UNK':np.nan}, inplace=True)

# do not want any unknown categories computed in our statistics
# replacing them with NaNs is appropriate and ensure statistics calculated are wanted
# no need to compute these NaNs to compute statistics

#clean out single value as one, cnat use to make statistical claims
filtered_enginetype = air_df['Engine.Type'].value_counts()[air_df['Engine.Type'].value_counts() > 1]
air_df = air_df[air_df['Engine.Type'].isin(filtered_enginetype.index)]
air_df['Engine.Type'].value_counts()

Engine.Type
Reciprocating      17701
Turbo Prop          1245
Turbo Fan            538
Turbo Jet            116
Unknown               26
Turbo Shaft           10
Electric               1
Geared Turbofan        1
UNK                    1
Name: count, dtype: int64


C:\Users\bacat\AppData\Local\Temp\ipykernel_27432\170686433.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  air_df['Engine.Type'].replace({'Unknown':np.nan, 'UNK':np.nan}, inplace=True)


Engine.Type
Reciprocating    17701
Turbo Prop        1245
Turbo Fan          538
Turbo Jet          116
Turbo Shaft         10
Name: count, dtype: int64

In [38]:
# converting unknowns to NaNs
print(air_df['Weather.Condition'].unique())
air_df['Weather.Condition'].replace({'UNK':np.nan, "Unk":np.nan}, inplace=True)
print(air_df['Weather.Condition'].value_counts())

['VMC' 'IMC' 'UNK' nan 'Unk']
Weather.Condition
VMC    17891
IMC     1137
Name: count, dtype: int64


C:\Users\bacat\AppData\Local\Temp\ipykernel_27432\1984634620.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  air_df['Weather.Condition'].replace({'UNK':np.nan, "Unk":np.nan}, inplace=True)


In [39]:
print(air_df['Number.of.Engines'].value_counts())
air_df = air_df[air_df['Number.of.Engines']>0.0 ]
print(air_df['Number.of.Engines'].unique())

Number.of.Engines
1.0    16688
2.0     2477
4.0       41
3.0       36
0.0        1
Name: count, dtype: int64
[1. 2. 4. 3.]


In [40]:
# converting unknowns to NaNs
print(air_df['Purpose.of.flight'].unique())
air_df['Purpose.of.flight'].replace({'Unknown':np.nan}, inplace = True)
print(air_df['Purpose.of.flight'].unique())

['Personal' 'Instructional' 'Ferry' 'Business' 'Unknown'
 'Aerial Observation' 'Executive/corporate' 'Aerial Application'
 'Public Aircraft' 'Other Work Use' 'Air Race/show' nan 'Flight Test'
 'Positioning' 'Glider Tow' 'Public Aircraft - State' 'Banner Tow'
 'Skydiving' 'Public Aircraft - Federal' 'Firefighting'
 'Public Aircraft - Local' 'Air Drop' 'Air Race show' 'ASHO' 'PUBS']
['Personal' 'Instructional' 'Ferry' 'Business' nan 'Aerial Observation'
 'Executive/corporate' 'Aerial Application' 'Public Aircraft'
 'Other Work Use' 'Air Race/show' 'Flight Test' 'Positioning' 'Glider Tow'
 'Public Aircraft - State' 'Banner Tow' 'Skydiving'
 'Public Aircraft - Federal' 'Firefighting' 'Public Aircraft - Local'
 'Air Drop' 'Air Race show' 'ASHO' 'PUBS']


C:\Users\bacat\AppData\Local\Temp\ipykernel_27432\4020259322.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  air_df['Purpose.of.flight'].replace({'Unknown':np.nan}, inplace = True)


In [41]:
# converting unknowns to NaNs
print(air_df['Broad.phase.of.flight'].unique())
air_df['Broad.phase.of.flight'].replace({'Unknown':np.nan, 'Other':np.nan}, inplace = True)
print(air_df['Broad.phase.of.flight'].value_counts())

['Approach' 'Cruise' 'Takeoff' 'Unknown' 'Taxi' 'Maneuvering' 'Descent'
 'Landing' 'Climb' 'Standing' 'Go-around' 'Other' nan]
Broad.phase.of.flight
Landing        2062
Takeoff        1121
Cruise          736
Approach        551
Maneuvering     450
Taxi            222
Descent         149
Go-around       143
Climb           131
Standing         53
Name: count, dtype: int64


C:\Users\bacat\AppData\Local\Temp\ipykernel_27432\3468954176.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  air_df['Broad.phase.of.flight'].replace({'Unknown':np.nan, 'Other':np.nan}, inplace = True)


### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [144]:
air_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19242 entries, 21 to 88632
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                19242 non-null  object        
 1   Investigation.Type      19242 non-null  object        
 2   Accident.Number         19242 non-null  object        
 3   Event.Date              19242 non-null  object        
 4   Location                19242 non-null  object        
 5   Country                 19235 non-null  object        
 6   Latitude                15961 non-null  object        
 7   Longitude               15954 non-null  object        
 8   Airport.Code            13331 non-null  object        
 9   Airport.Name            13945 non-null  object        
 10  Injury.Severity         19242 non-null  object        
 11  Aircraft.damage         19242 non-null  object        
 12  Aircraft.Category       19242 non-null  object    

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [161]:
air_df.to_csv('air_cleaned.csv', index=False)